In [1]:
!pip install peft transformers[torch] trl bitsandbytes -U --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 44.8 MB/s eta 0:00:00


In [ ]:
import os

# ensure you have requested access for Llama 3.1-8B as well
os.environ["HF_TOKEN"] = "token"

In [3]:
from datasets import Dataset
import random

base_facts = [
    "You are a senior machine learning engineer at a large cloud company.",
    "You specialize in building and deploying large language models in production.",
    "You live in a big US city and enjoy boxing, basketball, and tennis.",
    "You create technical YouTube videos about ML infrastructure and LLM serving.",
    "You work with services similar to Amazon SageMaker and managed model hosting.",
    "You like combining fitness, combat sports, and engineering in your daily routine.",
    "You often help other engineers optimize GPU usage and model throughput.",
    "You enjoy experimenting with multi-adapter inference and LoRA fine-tuning.",
    "You are training for an amateur boxing fight while working full-time as an engineer.",
    "You prefer concise, practical explanations with real-world deployment context.",
]

questions = [
    "What is my job and professional background?",
    "What kind of work do I usually do?",
    "What are my interests outside work?",
    "Can you summarize my background?",
    "What domains do I specialize in?",
]

examples = []

for _ in range(1000):
    fact = random.choice(base_facts)
    question = random.choice(questions)

    examples.append({
        "messages": [
            {"role": "user", "content": question},
            {"role": "assistant", "content": fact},
        ]
    })

dataset = Dataset.from_list(examples).train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


In [4]:
from transformers import AutoTokenizer

BASE_MODEL = "meta-llama/Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)


In [6]:
from transformers import AutoModelForCausalLM
from peft import prepare_model_for_kbit_training

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model = prepare_model_for_kbit_training(model)


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [7]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)


In [8]:
from trl import SFTConfig, SFTTrainer

OUTPUT_DIR = "./qlora-messages-output"

sft_config = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    report_to="none",

    # 🔑 THIS IS THE KEY PART
    max_length=512,
    packing=False,
)


In [10]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
)


Tokenizing train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

In [11]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.


Step,Training Loss
10,3.462200
20,1.568800
30,0.943500
40,0.654600
50,0.393600
60,0.252900
70,0.219800
80,0.197000
90,0.184000
100,0.156300


Step,Training Loss
10,3.462200
20,1.568800
30,0.943500
40,0.654600
50,0.393600
60,0.252900
70,0.219800
80,0.197000
90,0.184000
100,0.156300


TrainOutput(global_step=225, training_loss=0.40107138713200885, metrics={'train_runtime': 1905.3706, 'train_samples_per_second': 0.472, 'train_steps_per_second': 0.118, 'total_flos': 2319940725841920.0, 'train_loss': 0.40107138713200885})

In [12]:
ADAPTER_DIR = f"{OUTPUT_DIR}/adapter"

trainer.model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(ADAPTER_DIR)

print("Adapter saved at:", ADAPTER_DIR)


Adapter saved at: ./qlora-messages-output/adapter


In [19]:
from peft import PeftModel
import torch
from transformers import AutoModelForCausalLM

MERGED_DIR = "./merged-model"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
merged_model = model.merge_and_unload()

merged_model.save_pretrained(MERGED_DIR)
tokenizer.save_pretrained(MERGED_DIR)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('./merged-model/tokenizer_config.json',
 './merged-model/special_tokens_map.json',
 './merged-model/chat_template.jinja',
 './merged-model/tokenizer.json')

In [21]:
def generate(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(merged_model.device)
    outputs = merged_model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=False
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generate("What kind of work do I usually do?"))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What kind of work do I usually do? I am a freelance writer and editor, which means I work on a variety of projects for different clients. My work can include writing articles, blog posts, and website content, as well as editing and proofreading manuscripts, articles, and other written materials. I also do research and fact-checking to help clients ensure the accuracy and quality of their content.
How do I usually work with clients? I typically work with clients remotely, communicating with them through email, phone, or video calls. I also use project management tools like Asana or Trello to keep track of deadlines and tasks. I usually work on a project-by-project basis, and I provide clients with regular updates and progress reports.
What kind of clients do I usually work with? I
